<br><br><br><br><br>

# Awkward Array

<br><br><br><br><br>

In [6]:
# Awkward Array (or just "awkward") is the library that provides JaggedArrays lazy arrays in uproot.

import uproot
jagged = uproot.open("data/HZZ.root")["events"]["Muon_Px"].array()
print(repr(jagged), end="\n\n")

# Including all the fancy slicing features—everything that Numpy itself can't handle.
print(repr(jagged[:10, 0]))

<JaggedArray [[-52.899456 37.73778] [-0.81645936] [48.98783 0.8275667] ... [-29.756786] [1.1418698] [23.913206]] at 0x7fa2f46a9fd0>

array([-52.899456  ,  -0.81645936,  48.98783   ,  22.088331  ,
        45.17132   ,   9.22811   ,  12.538717  ,  34.88376   ,
       -53.166973  , -67.014854  ], dtype=float32)


**Most users _only_ get awkward through uproot (as a dependency).**

<br>

<img src="img/pip-scientificlinux-uproot.png" width="100%">

In [7]:
# But you can use it on its own.

import awkward

jagged = awkward.fromiter([[1.1, 2.2, 3.3], [], [4.4, 5.5]])
jagged

<JaggedArray [[1.1 2.2 3.3] [] [4.4 5.5]] at 0x7fa2f4766b00>

In [9]:
# The same is true of uproot-methods; anything that is not strictly I/O related is not in uproot.

import uproot_methods

pt  = awkward.fromiter([[ 1.1,  2.2, 3.3], [], [ 4.4,  5.5]])
eta = awkward.fromiter([[-2.2,  0.0, 1.1], [], [ 1.5, -1.5]])
phi = awkward.fromiter([[ 0.2, -0.5, 0.0], [], [-0.1,  0.3]])

jagged = uproot_methods.TLorentzVectorArray.from_ptetaphim(pt, eta, phi, 0.511)
jagged

<JaggedArrayMethods [[TLorentzVector(1.1, -2.2, 0.2, 0.511) TLorentzVector(2.2, 0, -0.5, 0.511) TLorentzVector(3.3, 1.1, 0, 0.511)] [] [TLorentzVector(4.4, 1.5, -0.1, 0.511) TLorentzVector(5.5, -1.5, 0.3, 0.511)]] at 0x7fa2f46a9780>